In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
%pip install "ultralytics<=8.3.40"
import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
import cv2
from IPython.display import Image as IPyImage
# Load the model
model_path = "/kaggle/input/yolov8s-docs/pytorch/text-documents/1/best.pt"
model = YOLO(model_path)

# Perform inference on an image
image_path = "/kaggle/input/signatures-and-stamps/1/MIn-oboronyi-RF-11-1-745x1024.jpg"  
results = model.predict(image_path)

# Display the results
for result in results:
    IPyImage(result,width=600)
    # Also, save the results to a file
    result.save(filename="output_image.jpg")  # Saves the output image with predictions

    # Print bounding box details
    for box in result.boxes:
        print(f"Class: {model.names[int(box.cls)]}, Confidence: {box.conf.item()}, Bounding Box: {box.xyxy}")

In [ ]:
IPyImage(filename="output_image.jpg",width=600)

In [ ]:
import os
import cv2
from ultralytics import YOLO

# Load the trained model
model_path = "/kaggle/input/yolov8s-docs/pytorch/text-documents/1/best.pt"
model = YOLO(model_path)

# Define valid classes (only process signatures & stamps)
VALID_CLASSES = {0: "signature", 1: "stamp"}

# Input dataset directory
input_dir = "/kaggle/input/signatures-and-stamps/1"

# Output directory for cropped images
output_dir = "/kaggle/working/cropped_images"
os.makedirs(output_dir, exist_ok=True)

# Collect all image files from dataset
image_paths = []
for root, _, files in os.walk(input_dir):
    for file in files:
        if file.lower().endswith((".jpg", ".jpeg", ".png")):  # Only process image files
            image_paths.append(os.path.join(root, file))

print(f"Found {len(image_paths)} images in dataset. Processing...")

# Process each image in the dataset
for i, image_path in enumerate(image_paths):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading {image_path}, skipping...")
        continue

    # Perform inference
    results = model.predict(image_path)

    # Process detected objects
    for j, result in enumerate(results):
        for box in result.boxes:
            class_id = int(box.cls)

            # Skip text & tables (Only keep stamps & signatures)
            if class_id not in VALID_CLASSES:
                continue

            # Get bounding box coordinates
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])

            # Crop the detected region
            cropped_img = image[y_min:y_max, x_min:x_max]

            # Save cropped image
            output_filename = f"{VALID_CLASSES[class_id]}_{i+1}_{j+1}.jpg"
            output_path = os.path.join(output_dir, output_filename)
            cv2.imwrite(output_path, cropped_img)

            print(f"Saved: {output_filename}")

print("✅ Cropping completed! Cropped images saved in:", output_dir)


In [ ]:
!zip -r cropped_signatures_stamps.zip /kaggle/working/cropped_images
